# Prototract Notebook - Kris Veselits

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc">
    <ul class="toc-item">
        <li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">Introduction</a></span>
            <ul class="toc-item">
                <li><span><a href="#USGS-Prototract-Tool" data-toc-modified-id="Description-1.1">USGS Prototract Tool</a></span>
                </li>
                <li><span><a href="#Tips" data-toc-modified-id="Tips-1.2">Tips</a></span>
                </li>
            </ul>
        </li>
        <li><span><a href="#Connect-to-your-GIS" data-toc-modified-id="Connect-to-your-GIS-2">Connect to your
                    GIS</a></span></li>
        <li><span><a href="#Loading-input-and-output-files" data-toc-modified-id="Enter-path-to-input-feature-3">Loading
                    Input File Options</a></span></li>
        <li><span><a href="#Setting-the-path-to-output-feature,-optional"
                    data-toc-modified-id="Setting-the-path-to-output-feature-optional-4">Setting the Path to Output
                    Feature - Optional</a></span></li>
        <li><span><a href="#Process:-Union" data-toc-modified-id="Process-Union-5">Process: Union</a></span></li>
        <li><span><a href="#Process:-Multipart-To-Singlepart"
                    data-toc-modified-id="Process:-Multipart-To-Singlepart-6">Process: Multipart to Singlepart</a></span>
        </li>
        <li><span><a href="#Process:-Aggregate-Polygons" data-toc-modified-id="#Process-Aggregate-Polygons-7">Process:
                    Aggregate Polygons</a></span></li>
        <li><span><a href="#Process:-Simplify-Polygon" data-toc-modified-id="#Process-Simplify-Polygon-8">Process:
                    Simpligy Polygon</a></span></li>
        <li><span><a href="#Process:-Smooth-Polygon" data-toc-modified-id="#Process-Smooth-Polygon-9">Process: Smooth
                    Polygon</a></span></li>
        <li><span><a href="#Process:-Buffer" data-toc-modified-id="#Process-Buffer-10">Process: Buffer</a></span></li>
        <li><span><a href="#Process:-Eliminate-Polygon-Part"
                    data-toc-modified-id="#Process-Eliminate-Polygon-Part-11">Process: Eliminate Polygon Part</a></span>
        </li>

</div>

# Introduction

## USGS Prototract Tool

TODO: Write tool description

## Tips


<ul><li>To view keyboard shortcuts, select "Help -> Keyboard Shortcuts" from the menu bar</li>
<li>After partially typing, press tab to bring up autocomplete when trying to remember commands/libraries</li>
<li>When trying to remember which parameters a method requires, press shift + tab within the paranthesis</li></ul>

# Connect to your GIS

## Important: Run the cell below first

Establish a connection to your organization which could be an ArcGIS Online organization or an ArcGIS Enterprise. To be able to run the code using ArcGIS API for Python, you would need to provide credentials of a user within an ArcGIS Online organization.

In [3]:
#File selection requires ipyfilechooser -- install from Python Command Prompt using:
#"pip install ipyfilechooser"

from arcgis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from contextlib import redirect_stdout
from ipywidgets import interactive
from IPython.display import display
from ipyfilechooser import FileChooser
import arcpy, os, io, pandas as pd
import ipywidgets as widgets

gis = GIS() # Create portal connection.

# Output path will default to the folder that contains the Notebook.ipynb
notebook_path = os.path.abspath("Prototract_Jupyter_KV.ipynb")
out_file = os.path.join(os.path.dirname(notebook_path), "out_file.shp")
Output_Finished = os.path.join(os.path.dirname(notebook_path), "Output_Finished.shp")

# function for displaying tool help
def help_string(tool, num, num1=0):
    f = io.StringIO()
    with redirect_stdout(f):
        out = ""
        help(tool)
        for line in (f.getvalue()).splitlines()[num1:num]:
            out += line + os.linesep
    return out

In [2]:
# To allow overwriting outputs change overwriteOutput option from False to True.
# If this is set to True, existing files with the same filename will be overwritten/lost.
arcpy.env.overwriteOutput = True

In [3]:
# Place cursor after method call and press shift + tab to see required parameters.
# Running this cell without specifying parameters will also display required parameters.

#help(arcpy.Union_analysis)
arcpy.Union_analysis

<function Union at 0x000001C35FB14B70>

# Loading input and output files

## 1. Select input file location

In [4]:
# Create and display a FileChooser widget
input_file = FileChooser()
display(input_file)

FileChooser(path='C:\Users\kjuli\PycharmProjects\USGS_Geostatistics\USGS_Geostatistics', filename='', show_hid…

In [5]:
# Print the selected path, filename, or both (Optional)
print(input_file.selected_path)
print(input_file.selected_filename)
print(input_file.selected)

in_file = str(input_file.selected)
print(in_file)

C:\Users\kjuli\PycharmProjects\USGS_Geostatistics\USGS_Geostatistics\Prototract\Input
Initial_tracts.shp
C:\Users\kjuli\PycharmProjects\USGS_Geostatistics\USGS_Geostatistics\Prototract\Input\Initial_tracts.shp
C:\Users\kjuli\PycharmProjects\USGS_Geostatistics\USGS_Geostatistics\Prototract\Input\Initial_tracts.shp


### 2. Select output file location (optional)
Note: Output path will default to the folder that contains the Notebook.ipynb unless the cell below is run.

In [6]:
# Create and display a FileChooser widget
output_file = FileChooser()
display(output_file)

FileChooser(path='C:\Users\kjuli\PycharmProjects\USGS_Geostatistics\USGS_Geostatistics', filename='', show_hid…

In [184]:
# Print the selected path, filename, or both
print(output_file.selected_path)
print(output_file.selected_filename)
print(output_file.selected)

out_file = str(output_file.selected)
print(out_file)

NameError: name 'output_file' is not defined

## Process: Union

### Gather Parameters
Run cell below and scroll down for interactive parameter selection

In [8]:
options=[
    ['Centimeters','DecimalDegrees', 'Decimeters', 'Feet', 'Inches', 'Kilometers',
    'Meters','Miles','Millimeters','NauticalMiles','Points', 'Yards','Unknown']]
style=dict(description_width='initial')

_dropdown = widgets.Dropdown()
_dropdown.value = None
_dropdown.options = ['All','NO_FID', 'ONLY_FID']
_dropdown.description = "Attributes To Join"
_dropdown.style = style

_checkbox = widgets.Checkbox()
_checkbox.value=False
_checkbox.description="Gaps Allowed"
_checkbox.indent=False

def f(a, b):
    display("Finished. Run cell below to continue.")
    return (str(a) + " " + str(b))

_linear_unit = interactive(f, {'manual': True}, a = widgets.IntText(
    value=None,
    description='Linear Unit Value',
    style=style
), b = widgets.Dropdown(
    options=options[0],
    value=None,
    description='Linear Unit Type',
    style=style
))

display(_checkbox)
display(_dropdown)
display(_linear_unit)

'Finished. Run cell below to continue.'

In [12]:
print(_checkbox.value), print(_dropdown.value), print(_linear_unit.result)

join_attributes = _dropdown.value

cluster_tolerance = _linear_unit.result
if("None" in cluster_tolerance):
    cluster_tolerance = None
    
gaps = _checkbox.value

# Place cursor at end of method name (arcpy.Union_analysis) and press "shift + tab" for help
arcpy.Union_analysis(in_file, out_file, join_attributes, cluster_tolerance, gaps)

<Result 'C:\\Users\\kjuli\\PycharmProjects\\USGS_Geostatistics\\USGS_Geostatistics\\out_file.shp'>

### Create map frame and plot points

In [13]:
import pandas as pd
sdf = pd.DataFrame.spatial.from_featureclass(out_file)
sdf.tail()
map1 = gis.map('California')
sdf.spatial.plot(map_widget= map1)
map1

MapView(layout=Layout(height='400px', width='100%'))

## Process: Multipart To Singlepart

In [14]:
# Place cursor at end of method name (arcpy.Union_analysis) and press "shift + tab" for help

Output_Singlepart = os.path.join(os.path.dirname(out_file), "Output_Singlepart.shp")
arcpy.MultipartToSinglepart_management(in_features=out_file, out_feature_class=Output_Singlepart)

<Result 'C:\\Users\\kjuli\\PycharmProjects\\USGS_Geostatistics\\USGS_Geostatistics\\Output_Singlepart.shp'>

## Update map frame and plot points

In [196]:
sdf = pd.DataFrame.spatial.from_featureclass(Output_Singlepart)
sdf.tail()
map1 = gis.map('California')
sdf.spatial.plot(map_widget= map1)
map1

MapView(layout=Layout(height='400px', width='100%'))

## Optional: delete intermediate files

In [195]:
# Process: Delete
arcpy.Delete_management(out_file, "")

<Result 'true'>

## Process: Aggregate Polygons

![Process: Aggregate Polygons](https://i.imgur.com/K9U3WZC.png)

In [15]:
Output_Aggregate = os.path.join(os.path.dirname(Output_Singlepart), "Output_Aggregate.shp")
Output_Table = os.path.join(os.path.dirname(Output_Singlepart), "Output_Aggregate_tbl.shp")

a = widgets.Text(
    value="0",
    description='Value:',
    style=dict(description_width='initial'),
    disabled=False
)

b = widgets.Dropdown(
    options=['Centimeters','DecimalDegrees', 'Decimeters', 'Feet', 'Inches', 'Kilometers',
             'Meters','Miles','Millimeters','NauticalMiles','Points', 'Unknown', 'Yards'],
    value='Centimeters',
    description='Units:',
    style=dict(description_width='initial'),
    disabled=False
)


c = widgets.Text(
    value="None",
    description='Value:',
    style=dict(description_width='initial'),
    disabled=False
)

d = widgets.Dropdown(
    options=['None','acres', 'ares','hectares', 'SquareCentimeters','SquareDecimeters',
             'SquareInches','SquareFeet','SquareKilometers','SquareMeters',
             'SquareMiles','SquareMillimeters','SquareYards','SquareMapUnits','unknown'],
    value='None',
    description='Units',
    style=dict(description_width='initial'),
    disabled=False
)

e = widgets.Text(
    value="None",
    description='Value:',
    style=dict(description_width='initial'),
    disabled=False
)

f = widgets.Dropdown(
    options=['None', 'acres', 'ares','hectares', 'SquareCentimeters','SquareDecimeters',
             'SquareInches','SquareFeet','SquareKilometers','SquareMeters',
             'SquareMiles','SquareMillimeters','SquareYards','SquareMapUnits','unknown'],
    value='None',
    description='Units:',
    style=dict(description_width='initial'),
    disabled=False
)

g = widgets.Checkbox(
    value=False,
    description='Preserve Orthogonal Shape',
    disabled=False,
    indent=False
)

# Create and display a FileChooser widget
print("Barrier Features (leave \"No file selected\" to not use)")
barrier = FileChooser()
display(barrier)

print("Aggregation Distance")
display(a)
display(b)
print("Minimum Area")
display(c)
display(d)
print("Minimum Hole Size")
display(e)
display(f)
display(g)

Barrier Features (leave "No file selected" to not use)


FileChooser(path='C:\Users\kjuli\PycharmProjects\USGS_Geostatistics\USGS_Geostatistics', filename='', show_hid…

Aggregation Distance


Text(value='0', description='Value:', style=DescriptionStyle(description_width='initial'))

Dropdown(description='Units:', options=('Centimeters', 'DecimalDegrees', 'Decimeters', 'Feet', 'Inches', 'Kilo…

...

Dropdown(description='Units', options=('None', 'acres', 'ares', 'hectares', 'SquareCentimeters', 'SquareDecime…

Minimum Hole Size


Text(value='None', description='Value:', style=DescriptionStyle(description_width='initial'))

Dropdown(description='Units:', options=('None', 'acres', 'ares', 'hectares', 'SquareCentimeters', 'SquareDecim…

Checkbox(value=False, description='Preserve Orthogonal Shape', indent=False)

In [20]:
Aggregation_Distance = a.value + " " + b.value

Minimum_Area = c.value + " " + d.value
if("None" in Minimum_Area):
    Minimum_Area = None

Minimum_Hole_Size = e.value + " " + f.value
if("None" in Minimum_Hole_Size):
    Minimum_Hole_Size = None

Preserve_orthogonal_shape = g.value

Barrier_Features = barrier.selected

# Place cursor at end of method name (arcpy.Union_analysis) and press "shift + tab" for help
arcpy.AggregatePolygons_cartography(
    in_features=Output_Singlepart, 
    out_feature_class=Output_Aggregate,
    aggregation_distance=Aggregation_Distance,
    minimum_area=Minimum_Area,
    minimum_hole_size=Minimum_Hole_Size,
    orthogonality_option=Preserve_orthogonal_shape,
    barrier_features=Barrier_Features)

<Result 'C:\\Users\\kjuli\\PycharmProjects\\USGS_Geostatistics\\USGS_Geostatistics\\Output_Aggregate.shp'>

## Update map frame and plot points

In [ ]:
sdf = pd.DataFrame.spatial.from_featureclass(Output_Aggregate)
sdf.tail()
map1 = gis.map('United States')
sdf.spatial.plot(map_widget= map1)
map1

## Optional: delete intermediate files

In [24]:
# Process: Delete
arcpy.Delete_management(Output_Singlepart, "")

<Result 'true'>

## Process: Simplify Polygon

![Process: Simplify Polygon](https://i.imgur.com/sfuA5H6.png)

In [11]:
Output_Simplify = os.path.join(os.path.dirname(notebook_path), "Output_Simplify.shp")

_Simplification_Algorithm = widgets.Dropdown(
    options=['POINT_REMOVE', 'BEND_SIMPLIFY', 'WEIGHTED_AREA', 'EFFECTIVE_AREA'],
    value='POINT_REMOVE',
    description='Attributes To Join:',
    style=dict(description_width='initial')
)

def g(a, b, c, d):
    display("Finished. Run cell below to continue.")
    return([a,b,c,d])

_Units = interactive(g, {'manual': True},a = widgets.IntText(
    value=0,
    description='Linear Unit Value',
    style=dict(description_width='initial')
), b = widgets.Dropdown(
    options=['Centimeters','DecimalDegrees', 'Decimeters', 'Feet', 'Inches', 'Kilometers',
             'Meters','Miles','Millimeters','NauticalMiles','Points', 'Unknown', 'Yards'],
    value='Unknown',
    description='Linear Unit Type:',
    style=dict(description_width='initial'),
), c = widgets.IntText(
    value=None,
    description='Value:',
    style=dict(description_width='initial'),
    disabled=False
),d = widgets.Dropdown(
    options=['acres', 'ares','hectares', 'SquareCentimeters','SquareDecimeters',
             'SquareInches','SquareFeet','SquareKilometers','SquareMeters',
             'SquareMiles','SquareMillimeters','SquareYards','SquareMapUnits','Unknown'],
    value='Unknown',
    description='Units:',
    style=dict(description_width='initial')
   ))

_Keep_collapsed_points = widgets.Checkbox(
    value=True,
    description='Keep collapsed points (Specifies whether to create an output point feature class to store the centers of polygons that are removed because they are smaller than the Minimum area parameter. The point output is derived; it will use the same name and location as the Output feature class parameter but with a _Pnt suffix)',
    indent=False
)

# Create and display a FileChooser widget
print("Barrier Features (leave \"No file selected\" to not use)")
barrier = FileChooser()
display(barrier)
display(_Simplification_Algorithm)
display(_Units)
display(_Keep_collapsed_points)

Barrier Features (leave "No file selected" to not use)


FileChooser(path='C:\Users\kjuli\PycharmProjects\USGS_Geostatistics\USGS_Geostatistics', filename='', show_hid…

Dropdown(description='Attributes To Join:', options=('POINT_REMOVE', 'BEND_SIMPLIFY', 'WEIGHTED_AREA', 'EFFECT…

interactive(children=(IntText(value=0, description='Linear Unit Value', style=DescriptionStyle(description_wid…

Checkbox(value=True, description='Keep collapsed points (Specifies whether to create an output point feature c…

In [37]:
Simplification_Algorithm = _Simplification_Algorithm.value

Simplification_Tolerance = str(_Units.result[0]) + " " + str(_Units.result[1])
if("None" in Simplification_Tolerance):
    Simplification_Tolerance = None

Minimum_Area_2 = str(_Units.result[2]) + " " + str(_Units.result[3])
if("None" in Minimum_Area_2):
    Minimum_Area_2 = None

Keep_collapsed_points = _Keep_collapsed_points.value

Input_Barrier_Layers = barrier.selected

# Place cursor at end of method name (arcpy.Union_analysis) and press "shift + tab" for help
arcpy.SimplifyPolygon_cartography(in_features=Output_Aggregate, out_feature_class=Output_Simplify,
                                      algorithm=Simplification_Algorithm, tolerance=Simplification_Tolerance,
                                      minimum_area=Minimum_Area_2,
                                      collapsed_point_option=Keep_collapsed_points, in_barriers=Input_Barrier_Layers)[0]

'C:\\Users\\kjuli\\PycharmProjects\\USGS_Geostatistics\\USGS_Geostatistics\\Output_Simplify.shp'

## Update map frame and plot points

In [ ]:
sdf = pd.DataFrame.spatial.from_featureclass(Output_Simplify)
sdf.tail()
map1 = gis.map('United States')
sdf.spatial.plot(map_widget= map1)
map1

## Optional: delete intermediate files

In [ ]:
# Process: Delete
arcpy.Delete_management(Output_Aggregate, "")

In [ ]:
# Process: Delete optional collapsed points
arcpy.Delete_management(Keep_collapsed_points, "")

## Process: Smooth Polygon

![Process: Smooth Polygon](https://i.imgur.com/J6o7bou.png)

In [4]:
Output_Smooth = os.path.join(os.path.dirname(notebook_path), "Output_Smooth.shp")

_Smoothing_Algorithm = widgets.Dropdown(
    options=['PAEK', 'BEZIER_INTERPOLATION'],
    value='PAEK',
    description='Smoothing Algorithm:',
    style=dict(description_width='initial')
)

def h(a, b):
    return(str(a) + " " + str(b))

_Smoothing_Tolerance = interactive(h, {'manual': True},a = widgets.IntText(
    value=1,
    description='Linear Unit Value',
    style=dict(description_width='initial')
), b = widgets.Dropdown(
    options=['Centimeters','DecimalDegrees', 'Decimeters', 'Feet', 'Inches', 'Kilometers',
             'Meters','Miles','Millimeters','NauticalMiles','Points', 'Unknown', 'Yards'],
    value='Unknown',
    description='Linear Unit Type:',
    style=dict(description_width='initial')
))

_Handling_Topological_Errors_2 = widgets.Dropdown(
    options=['NO_CHECK', 'FLAG_ERRORS', 'RESOLVE_ERRORS'],
    value='NO_CHECK',
    description='Handling Topological Errors:',
    style=dict(description_width='initial')
)

# Create and display a FileChooser widget
print("Barrier Features (leave \"No file selected\" to not use)")
barrier = FileChooser()
display(barrier)

display(_Smoothing_Algorithm)
display(_Handling_Topological_Errors_2)
print("Smoothing Tolerance")
display(_Smoothing_Tolerance)

Barrier Features (leave "No file selected" to not use)


FileChooser(path='C:\Users\kjuli\PycharmProjects\USGS_Geostatistics\USGS_Geostatistics', filename='', show_hid…

Dropdown(description='Smoothing Algorithm:', options=('PAEK', 'BEZIER_INTERPOLATION'), style=DescriptionStyle(…

Dropdown(description='Handling Topological Errors:', options=('NO_CHECK', 'FLAG_ERRORS', 'RESOLVE_ERRORS'), st…

Smoothing Tolerance


interactive(children=(IntText(value=1, description='Linear Unit Value', style=DescriptionStyle(description_wid…

In [11]:
Smoothing_Algorithm = _Smoothing_Algorithm.value

Smoothing_Tolerance = _Smoothing_Tolerance.result
if("None" in Smoothing_Tolerance):
    Smoothing_Tolerance = None

Handling_Topological_Errors_2 = _Handling_Topological_Errors_2.value

Input_Barrier_Layers_2_ = barrier.selected

# Place cursor at end of method name (arcpy.Union_analysis) and press "shift + tab" for help
arcpy.SmoothPolygon_cartography(in_features=Output_Simplify, out_feature_class=Output_Smooth,
                                    algorithm=Smoothing_Algorithm, tolerance=Smoothing_Tolerance,
                                    error_option=Handling_Topological_Errors_2, in_barriers=Input_Barrier_Layers_2_)

<Result 'C:\\Users\\kjuli\\PycharmProjects\\USGS_Geostatistics\\USGS_Geostatistics\\Output_Smooth.shp'>

## Update map frame and plot points

In [ ]:
sdf = pd.DataFrame.spatial.from_featureclass(Output_Smooth)
sdf.tail()
map1 = gis.map('United States')
sdf.spatial.plot(map_widget= map1)
map1

## Optional: delete intermediate files

In [ ]:
# Process: Delete
arcpy.Delete_management(Output_Simplify, "")

## Process: Buffer

![Process: Buffer](https://i.imgur.com/G5hFZAo.png)

In [80]:
Output_Buffer = os.path.join(os.path.dirname(notebook_path), "Output_Buffer.shp")

_Side_Type = widgets.Dropdown(
    options=['FULL', 'LEFT', 'RIGHT','OUTSIDE_ONLY'],
    value='FULL',
    description='Side Type:',
    style=dict(description_width='initial')
)

_End_Type = widgets.Dropdown(
    options=['ROUND', 'FLAT'],
    value='ROUND',
    description='End Type:',
    style=dict(description_width='initial')
)


_Method = widgets.Dropdown(
    options=['PLANAR', 'GEODESIC'],
    value='PLANAR',
    description='Method:',
    style=dict(description_width='initial')
)

_Dissolve_Type = widgets.Dropdown(
    options=['None', 'ALL', 'LIST'],
    value='None',
    description='Dissolve Type:',
    style=dict(description_width='initial')
)

def i(a, b):
    display("Finished. Run cell below to continue.")
    return (str(a) + " " + str(b))

_Distance_value_or_field = interactive(i, {'manual': True},a = widgets.IntText(
    value=0,
    description='Linear Unit Value',
    style=dict(description_width='initial')
), b = widgets.Dropdown(
    options=['None','Centimeters','DecimalDegrees', 'Decimeters', 'Feet', 'Inches', 'Kilometers',
             'Meters','Miles','Millimeters','NauticalMiles','Points', 'Unknown', 'Yards'],
    value='Unknown',
    description='Linear Unit Type:',
    style=dict(description_width='initial'),
))

display(_Side_Type)
display(_End_Type)
display(_Method)
display(_Dissolve_Type)
display(_Distance_value_or_field)

'Finished. Run cell below to continue.'

In [82]:
Distance_value_or_field = _Distance_value_or_field.result
if("None" in Distance_value_or_field):
    Distance_value_or_field = None
    
Side_Type = _Side_Type.value
if("None" in Side_Type):
    Side_Type = None

End_Type = _End_Type.value
Dissolve_Type = _Dissolve_Type.value
if("None" in Dissolve_Type):
    Dissolve_Type = None
Method = _Method.value

# Place cursor at end of method name (arcpy.Union_analysis) and press "shift + tab" for help
arcpy.Buffer_analysis(in_features=Output_Smooth, out_feature_class=Output_Buffer,
                          buffer_distance_or_field=Distance_value_or_field, line_side=Side_Type,
                          line_end_type=End_Type, dissolve_option=Dissolve_Type,
                          method=Method)

KeyboardInterrupt: 

## Update map frame and plot points

In [ ]:
sdf = pd.DataFrame.spatial.from_featureclass(Output_Buffer)
sdf.tail()
map1 = gis.map('United States')
sdf.spatial.plot(map_widget= map1)
map1

## Optional: delete intermediate files

In [ ]:
# Process: Delete
arcpy.Delete_management(Output_Smooth, "")

## Process: Eliminate Polygon Part

![Process: Eliminate Polygon Part](https://i.imgur.com/ngPe4NU.png)

In [52]:
def f(a, b):
    display("Finished. Run cell below to continue.")
    return ([a,b])

_Condition = interactive(f, {'manual': True}, a = widgets.Dropdown(
    options=['AREA', 'PERCENT', 'AREA_AND_PERCENT','AREA_OR_PERCENT'],
    value=None,
    description='Condition:',
    style=dict(description_width='initial')
), b = widgets.IntText(
    value=None,
    description='Percentage:',
    style=dict(description_width='initial')
))


_Eliminate_contained_parts_only = widgets.Checkbox()
_Eliminate_contained_parts_only.value=False
_Eliminate_contained_parts_only.description="Eliminate Contained Parts Only"
_Eliminate_contained_parts_only.indent=False

display(_Condition)
display(_Eliminate_contained_parts_only)

'Finished. Run cell below to continue.'

In [54]:
Condition = _Condition.result[0]
Area = _Condition.result[1]
Percentage = _Condition.result[1]
Eliminate_contained_parts_only = _Eliminate_contained_parts_only.value

# Place cursor at end of method name (arcpy.Union_analysis) and press "shift + tab" for help
arcpy.EliminatePolygonPart_management(in_features=Output_Buffer, out_feature_class=Output_Finished,
                                          condition=Condition, part_area=Area, part_area_percent=Percentage,
                                          part_option=Eliminate_contained_parts_only)

<Result 'C:\\Users\\kjuli\\PycharmProjects\\USGS_Geostatistics\\USGS_Geostatistics\\Output_Finished.shp'>

## Update map frame and plot points

In [ ]:
sdf = pd.DataFrame.spatial.from_featureclass(Output_Finished)
sdf.tail()
map1 = gis.map('United States')
sdf.spatial.plot(map_widget= map1)
map1

## Optional: delete intermediate files

In [ ]:
# Process: Delete
arcpy.Delete_management(Output_Buffer, "")